This is a simple demonstration of Debiased Machine Learning estimator for the Conditional Average Treatment Effect. 
Goal is to estimate the effect of 401(k) eligibility on net financial assets for each value of income. 
Data set is the same as in (Chernozhukov, Hansen, 2004). 


The method is based on the following paper. 

Title:  Debiased Machine Learning of Conditional Average Treatment Effect and Other Causal Functions

Authors: Semenova, Vira and Chernozhukov, Victor. 

Arxiv version: https://arxiv.org/pdf/1702.06240.pdf

Published version with replication code: https://academic.oup.com/ectj/advance-article/doi/10.1093/ectj/utaa027/5899048


[1]Victor Chernozhukov and Christian Hansen. The impact of 401(k) participation on the wealth distribution: An instrumental quantile regression analysis. Review of Economics and Statistics, 86(3):735–751, 2004.

Background

The target function is Conditional Average Treatment Effect, defined as 

$$ g(x)=E [ Y(1) - Y(0) |X=x], $$ 

where $Y(1)$ and $Y(0)$ are potential outcomes in treated and control group. In our case, $Y(1)$ is the potential Net Financial Assets if a subject is eligible for 401(k), and $Y(0)$ is the potential Net Financial Assets if a subject is ineligible. $X$ is a covariate of interest, in this case, income.
$ g(x)$ shows expected effect of eligibility on NET TFA for a subject whose income level is $x$.



If eligibility indicator is independent of $Y(1), Y(0)$, given pre-401-k assignment characteristics $Z$, the function can expressed in terms of observed data (as opposed to hypothetical, or potential outcomes). Observed data consists of  realized NET TFA $Y = D Y(1) + (1-D) Y(0)$, eligibility indicator $D$, and covariates $Z$ which includes $X$, income. The expression for $g(x)$ is

$$ g(x) = E [ Y (\eta_0) \mid X=x], $$
where the transformed outcome variable is

$$Y (\eta) = \dfrac{D}{s(Z)} \left( Y - \mu(1,Z) \right) - \dfrac{1-D}{1-s(Z)} \left( Y - \mu(0,Z) \right) + \mu(1,Z) - \mu(0,Z),$$

the probability of eligibility is 

$$s_0(z) = Pr (D=1 \mid Z=z),$$ 

the expected net financial asset given $D =d \in \{1,0\}$ and $Z=z$ is

$$ \mu(d,z) = E[ Y \mid Z=z, D=d]. $$

Our goal is to estimate $g(x)$.


In step 1, we estimate the unknown functions $s_0(z),  \mu(1,z),  \mu(0,z)$ and plug them into $Y (\eta)$.


In step 2, we approximate the function $g(x)$ by a linear combination of basis functions:

$$ g(x) = p(x)' \beta_0, $$


where $p(x)$ is a vector of polynomials or splines and

$$ \beta_0 = (E p(X) p(X))^{-1} E p(X) Y (\eta_0) $$

is the best linear predictor. We report

$$
\widehat{g}(x) = p(x)' \widehat{\beta},
$$

where $\widehat{\beta}$ is the ordinary least squares estimate of $\beta_0$ defined on the random sample $(X_i, D_i, Y_i)_{i=1}^N$

$$
	\widehat{\beta} :=\left( \dfrac{1}{N} \sum_{i=1}^N p(X_i) p(X_i)' \right)^{-1} \dfrac{1}{N} \sum_{i=1}^N  p(X_i)Y_i(\widehat{\eta})
$$

In [ ]:
import numpy as np
import pandas as pd
import doubleml as dml
from doubleml.datasets import fetch_401K

# Import relevant packages
import pyreadr
from sklearn import preprocessing
import patsy

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from xgboost import XGBClassifier, XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline


In [ ]:
# pension = fetch_401K(return_type='DataFrame')

In [ ]:
pension_Read = pyreadr.read_r("../data/pension.Rdata")
pension = pension_Read[ 'pension' ]

In [ ]:
pension["net_tfa"] = pension["net_tfa"] / 1000
pension

In [ ]:
## covariate of interest -- log income --
pension["inc"] = np.log(pension["inc"])

pension = pension[~pension.isin([np.nan, np.inf, -np.inf]).any(1)]
pension = pension.reset_index()

In [ ]:
## outcome variable -- total net financial assets
Y = pension["net_tfa"]

## binary treatment --  indicator of 401(k) eligibility
D = pension["e401"]

X = pension["inc"]

## raw covariates so that Y(1) and Y(0) are independent of D given Z
Z = pension[["age","inc","fsize","educ","male","db","marr","twoearn","pira","hown","hval","hequity","hmort",
              "nohs","hs","smcol"]]

y_name = "net_tfa"
d_name = "e401"
form_z = "(poly(age, 6) + poly(inc, 8) + poly(educ, 4) + poly(fsize,2) + as.factor(marr) + as.factor(twoearn) + as.factor(db) + as.factor(pira) + as.factor(hown))^2"

In [ ]:
print("\n sample size is {} \n".format(len(Y)))
print("\n num raw covariates z is {} \n".format(Z.shape[1]))

In [ ]:
features = pension.copy()[['marr', 'twoearn', 'db', 'pira', 'hown']]


In [ ]:
poly_dict = {'age': 6,
             'inc': 8,
             'educ': 4,
             'fsize': 2}
for key, degree in poly_dict.items():
    poly = PolynomialFeatures(degree, include_bias=False)
    data_transf = poly.fit_transform(pension[[key]])
    x_cols = poly.get_feature_names([key])
    data_transf = pd.DataFrame(data_transf, columns=x_cols)
    
    features = pd.concat((features, data_transf),
                          axis=1, sort=False)

In [ ]:
import patsy 
from patsy import ModelDesc, Term, EvalFactor

In [ ]:
features

In [ ]:
new_columns = ['marr', 'twoearn', 'db', 'pira', 'hown',\
        'age', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', \
        'inc', 'inc_2', 'inc_3', 'inc_4', 'inc_5', 'inc_6', 'inc_7', 'inc_8', \
        'educ', 'educ_2', 'educ_3', 'educ_4',\
        'fsize', 'fsize_2']

In [ ]:
features.columns = [new_columns]
features

In [ ]:
formula = "(marr + twoearn + db + pira + hown + \
         age + age_2 +  age_3 +  age_4 +  age_5 +  age_6 + \
         inc +  inc_2 +  inc_3 + inc_4 +  inc_5 +  inc_6 +  inc_7 +  inc_8 + \
         educ +  educ_2 + educ_3 + educ_4 + \
         fsize +  fsize_2)**2"
formula

In [ ]:
y_name = pension["net_tfa"].to_numpy()
d_name = pension["e401"].to_numpy()
form_z = patsy.dmatrix(formula, features)


In [ ]:
Cs = 0.0001*np.logspace(0, 4, 10)
clf = LogisticRegressionCV(cv=5, penalty='l1', solver='liblinear',
                                                 Cs = Cs, max_iter=100).fit(form_z, d_name)

In [ ]:
a =  clf.predict(form_z)
np.var(a)

In [ ]:
np.logspace(0, 4, 10)